In [41]:
import pandas as pd
import numpy as numpy
import json 
import os 
import pylab as pl
import urllib
import requests
import geopandas as gp
import fiona
import shapely
from fiona.crs import from_epsg
import os
import pandas as pd
import urllib
import requests
import json
import sys
import io
import shapely
import base64
import urllib.request
import functools
import geopandas as gp
import fiona
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/katharinevoorhees/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['nan']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [42]:
myAPI = '90341a28f02794c77f6e3abb153a4952a0bb480b'

In [43]:
#pull in library of variable names
url = "https://api.census.gov/data/2016/acs/acs5/variables.json"
resp = requests.request('GET', url)
aff1y = json.loads(resp.text)

affkeys = np.array(list(aff1y['variables'].keys()))

In [44]:
#search for commute codes
[(k, aff1y['variables'][k]['label'])  for k in affkeys if k.startswith ('B08301') ]

[('B08301_007E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 4-person carpool'),
 ('B08301_006E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 3-person carpool'),
 ('B08301_009E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 7-or-more-person carpool'),
 ('B08301_008E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 5- or 6-person carpool'),
 ('B08301_003E', 'Estimate!!Total!!Car, truck, or van!!Drove alone'),
 ('B08301_002E', 'Estimate!!Total!!Car, truck, or van'),
 ('B08301_005E',
  'Estimate!!Total!!Car, truck, or van!!Carpooled!!In 2-person carpool'),
 ('B08301_004E', 'Estimate!!Total!!Car, truck, or van!!Carpooled'),
 ('B08301_011E',
  'Estimate!!Total!!Public transportation (excluding taxicab)!!Bus or trolley bus'),
 ('B08301_010E', 'Estimate!!Total!!Public transportation (excluding taxicab)'),
 ('B08301_013E',
  'Estimate!!Total!!Public transportation (excluding taxicab)!!Subway or elevated'),
 ('B08301_012E',
  'Estimate!!Total!!Public transpor

In [45]:
#dictionary for renaming later
column_keys= {'B08301_001E':'total_total','B08301_010E':'total_public_transpo',
        'B08301_011E':'bus','B08301_012E':'streetcar','B08301_013E':'subway',
        'B08301_002E': 'total_car_van_truck', 'B08301_003E':'car_vt_alone',
        'B08301_004E':'car_vt_carpooled', 'B08301_018E':'bicycle','B08301_019E':'walked',
        'B08301_014E':'railroad', 'B08301_015E':'ferry','B08301_016E':'taxi',
        'B08301_017E':'motorcycle','B08301_021E':'worked_at_home', 'B08301_020E':'other_means'}

In [46]:
#name keys
key1 = 'B08301_001E,B08301_010E,B08301_011E,B08301_012E,B08301_013E,B08301_002E,B08301_003E,'
key2 = 'B08301_004E,B08301_018E,B08301_019E,B08301_014E,B08301_015E,B08301_016E,B08301_017E,'
key3 = 'B08301_021E,B08301_020E'

In [47]:
#download info from census
year = '2017'

counties = [ 5, 47, 61, 81, 85]

url = "https://api.census.gov/data/" + year + "/acs/acs5?get=" +key1+key2+key3+\
    ",NAME&for=tract:*&in=state:36&in=county:*&key=" + myAPI
resp = requests.request('GET', url).content
commute = pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

commute = commute[commute['county'].isin(counties)]


commute.head(1)

,B08301_001E,B08301_010E,B08301_011E,B08301_012E,B08301_013E,B08301_002E,B08301_003E,B08301_004E,B08301_018E,B08301_019E,B08301_014E,B08301_015E,B08301_016E,B08301_017E,B08301_021E,B08301_020E,NAME,state,county,tract,Unnamed: 20
0,936,652,49,0,603,118,89,29,13,91,0,0,0,5,57,0,"Census Tract 119, Queens County, New York",36,81,11900,NaN


In [48]:
commute.rename(columns=column_keys, inplace=True)

In [49]:
commute.head(3)

,total_total,total_public_transpo,bus,streetcar,subway,total_car_van_truck,car_vt_alone,car_vt_carpooled,bicycle,walked,railroad,ferry,taxi,motorcycle,worked_at_home,other_means,NAME,state,county,tract,Unnamed: 20
0,936,652,49,0,603,118,89,29,13,91,0,0,0,5,57,0,"Census Tract 119, Queens County, New York",36,81,11900,NaN
1,733,241,45,0,192,387,338,49,0,35,4,0,7,22,41,0,"Census Tract 135, Queens County, New York",36,81,13500,NaN
2,1024,565,71,0,457,296,253,43,0,64,37,0,21,13,61,4,"Census Tract 136, Queens County, New York",36,81,13600,NaN


In [50]:
#pull in previously made key
os.chdir('/Users/katharinevoorhees/CAPSTONE/')
zones = pd.read_csv('better_taxizone_censustract_key.csv')
zones.head()

,index_right,OBJECTID,zone,tractid,bcode,tractnum
0,1968,2.0,Jamaica Bay,36081107202,36081,107202
1,222,3.0,Allerton/Pelham Gardens,36005031000,36005,31000
2,224,3.0,Allerton/Pelham Gardens,36005031400,36005,31400
3,226,3.0,Allerton/Pelham Gardens,36005031800,36005,31800
4,225,3.0,Allerton/Pelham Gardens,36005031600,36005,31600


In [51]:
len(zones['tractnum'].unique())

1332

In [52]:
len(commute['tract'].unique())

1335

In [53]:
#get country codes for join
zones['bcode'] = zones['bcode'].astype('str')
zones['cnty'] = zones['bcode'].str[-3:].astype('int')
zones.head()

,index_right,OBJECTID,zone,tractid,bcode,tractnum,cnty
0,1968,2.0,Jamaica Bay,36081107202,36081,107202,81
1,222,3.0,Allerton/Pelham Gardens,36005031000,36005,31000,5
2,224,3.0,Allerton/Pelham Gardens,36005031400,36005,31400,5
3,226,3.0,Allerton/Pelham Gardens,36005031800,36005,31800,5
4,225,3.0,Allerton/Pelham Gardens,36005031600,36005,31600,5


In [54]:
joined = commute.merge(zones, left_on=['county','tract'], right_on=['cnty','tractnum'])

joined.shape

(2161, 28)

In [55]:
#see if any tracts don't have a taxi zone
nan = joined[joined['OBJECTID'].isnull()]
nan

,total_total,total_public_transpo,bus,streetcar,subway,total_car_van_truck,car_vt_alone,car_vt_carpooled,bicycle,walked,railroad,ferry,taxi,motorcycle,worked_at_home,other_means,NAME,state,county,tract,Unnamed: 20,index_right,OBJECTID,zone,tractid,bcode,tractnum,cnty


In [56]:
#aggregate by zone
commute_taxi_zone = joined.groupby('OBJECTID').sum().reset_index()

In [57]:
commute_taxi_zone.columns

Index(['OBJECTID', 'total_total', 'total_public_transpo', 'bus', 'streetcar',
       'subway', 'total_car_van_truck', 'car_vt_alone', 'car_vt_carpooled',
       'bicycle', 'walked', 'railroad', 'ferry', 'taxi', 'motorcycle',
       'worked_at_home', 'other_means', 'state', 'county', 'tract',
       'Unnamed: 20', 'index_right', 'tractid', 'tractnum', 'cnty'],
      dtype='object')

In [58]:
commute_taxi_zone = commute_taxi_zone.loc[:,'OBJECTID':'other_means']

In [59]:
commute_taxi_zone.rename(columns={'OBJECTID':'taxi_zone'}, inplace=True)

commute_taxi_zone.head(2)

,taxi_zone,total_total,total_public_transpo,bus,streetcar,subway,total_car_van_truck,car_vt_alone,car_vt_carpooled,bicycle,walked,railroad,ferry,taxi,motorcycle,worked_at_home,other_means
0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3.0,14002,5991,1285,40,4399,6656,5643,1013,7,738,267,0,39,27,491,53


In [60]:
commute_taxi_zone.columns

Index(['taxi_zone', 'total_total', 'total_public_transpo', 'bus', 'streetcar',
       'subway', 'total_car_van_truck', 'car_vt_alone', 'car_vt_carpooled',
       'bicycle', 'walked', 'railroad', 'ferry', 'taxi', 'motorcycle',
       'worked_at_home', 'other_means'],
      dtype='object')

In [61]:
commute_taxi_zone[commute_taxi_zone['taxi_zone'] == 263][['total_total','total_public_transpo','walked','taxi']]

,total_total,total_public_transpo,walked,taxi
260,31069,21933,3836,1073


In [96]:
commute_taxi_zone.total_total.sum()

3975060

In [97]:
commute_taxi_zone.sum(axis = 0, skipna = True)

taxi_zone                 34611.0
total_total             3975060.0
total_public_transpo    2247686.0
bus                      423349.0
streetcar                  8822.0
subway                  1745407.0
total_car_van_truck     1057870.0
car_vt_alone             874845.0
car_vt_carpooled         183025.0
bicycle                   47047.0
walked                   396134.0
railroad                  59227.0
ferry                     10881.0
taxi                      38653.0
motorcycle                 3238.0
worked_at_home           162537.0
other_means               21895.0
dtype: float64

In [106]:
for i in commute_taxi_zone.columns: 
    prop = commute_taxi_zone[i].sum()/commute_taxi_zone.total_total.sum()
    round_prop = np.round((prop*100),2)
    print('The true proportion of {} is {}'.format(i,round_prop))

The true proportion of taxi_zone is 0.87
The true proportion of total_total is 100.0
The true proportion of total_public_transpo is 56.54
The true proportion of bus is 10.65
The true proportion of streetcar is 0.22
The true proportion of subway is 43.91
The true proportion of total_car_van_truck is 26.61
The true proportion of car_vt_alone is 22.01
The true proportion of car_vt_carpooled is 4.6
The true proportion of bicycle is 1.18
The true proportion of walked is 9.97
The true proportion of railroad is 1.49
The true proportion of ferry is 0.27
The true proportion of taxi is 0.97
The true proportion of motorcycle is 0.08
The true proportion of worked_at_home is 4.09
The true proportion of other_means is 0.55


In [107]:
eventual = commute_taxi_zone[['taxi_zone','total_public_transpo','walked','taxi', 'total_car_van_truck']]
eventual['temp_sum'] = eventual['total_public_transpo'] + eventual['walked'] + eventual['taxi'] +  eventual['total_car_van_truck']

/Users/katharinevoorhees/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [109]:
for i in eventual.columns: 
    prop = eventual[i].sum()/eventual.temp_sum.sum()
    round_prop = np.round((prop*100),2)
    print('The inflated proportion of {} for our model is {}'.format(i,round_prop))

The inflated proportion of taxi_zone for our model is 0.93
The inflated proportion of total_public_transpo for our model is 60.09
The inflated proportion of walked for our model is 10.59
The inflated proportion of taxi for our model is 1.03
The inflated proportion of total_car_van_truck for our model is 28.28
The inflated proportion of temp_sum for our model is 100.0


In [111]:
 stat = eventual.temp_sum.sum()/commute_taxi_zone.total_total.sum()
print('We are accounting for {} percent of ACS commuters with our modes'.format(np.round(stat*100)))

We are accounting for 94.0 percent of ACS commuters with our modes


In [62]:
new = commute_taxi_zone[['taxi_zone','total_public_transpo','walked','taxi', 'total_car_van_truck']]
new['temp_sum'] = new['total_public_transpo'] + new['walked'] + new['taxi'] +  new['total_car_van_truck']
col = ['total_public_transpo','walked','taxi', 'total_car_van_truck']
for i in col: `
    new[i] = new[i]/new['temp_sum']
new.head()

/Users/katharinevoorhees/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/katharinevoorhees/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,taxi_zone,total_public_transpo,walked,taxi,total_car_van_truck,temp_sum
0,2.0,NaN,NaN,NaN,NaN,0
1,3.0,0.446290,0.054976,0.002905,0.495828,13424
2,4.0,0.653460,0.247877,0.025276,0.073387,10247
3,5.0,0.226116,0.004921,0.001867,0.767097,11786
4,6.0,0.374791,0.037601,0.001827,0.585782,6569


In [64]:
os.chdir('/Users/katharinevoorhees/Downloads/')
t_nest = pd.read_csv('taxi_proportions.csv')[['PUlocationID','Taxi proportion','FHV proportion','FHV_shared proportion']]
t_nest.head()

,PUlocationID,Taxi proportion,FHV proportion,FHV_shared proportion
0,1.0,0.324470,0.599125,0.076405
1,2.0,0.224490,0.704082,0.071429
2,3.0,0.004564,0.705461,0.289975
3,4.0,0.159682,0.525024,0.315294
4,5.0,0.005298,0.856656,0.138046


In [65]:
new = new.merge(t_nest, left_on='taxi_zone', right_on='PUlocationID')

In [66]:
new = new.rename(columns={'Taxi proportion':'new_taxi','FHV proportion':'FHV', 'FHV_shared proportion':'FHV_share'})

In [67]:
new.dropna(inplace=True)
new.head()

,taxi_zone,total_public_transpo,walked,taxi,total_car_van_truck,temp_sum,PUlocationID,new_taxi,FHV,FHV_share
1,3.0,0.446290,0.054976,0.002905,0.495828,13424,3.0,0.004564,0.705461,0.289975
2,4.0,0.653460,0.247877,0.025276,0.073387,10247,4.0,0.159682,0.525024,0.315294
3,5.0,0.226116,0.004921,0.001867,0.767097,11786,5.0,0.005298,0.856656,0.138046
4,6.0,0.374791,0.037601,0.001827,0.585782,6569,6.0,0.011326,0.793525,0.195148
5,7.0,0.775097,0.073059,0.005607,0.146236,42445,7.0,0.124065,0.693247,0.182689


In [68]:
new['new_taxi'] = new['taxi'] * new['new_taxi']
new['FHV'] = new['FHV'] * new['taxi']
new['FHV_share'] = new['FHV_share'] * new['taxi']
new.head()

,taxi_zone,total_public_transpo,walked,taxi,total_car_van_truck,temp_sum,PUlocationID,new_taxi,FHV,FHV_share
1,3.0,0.446290,0.054976,0.002905,0.495828,13424,3.0,0.000013,0.002050,0.000842
2,4.0,0.653460,0.247877,0.025276,0.073387,10247,4.0,0.004036,0.013270,0.007969
3,5.0,0.226116,0.004921,0.001867,0.767097,11786,5.0,0.000010,0.001599,0.000258
4,6.0,0.374791,0.037601,0.001827,0.585782,6569,6.0,0.000021,0.001450,0.000356
5,7.0,0.775097,0.073059,0.005607,0.146236,42445,7.0,0.000696,0.003887,0.001024


In [69]:
new.columns

Index(['taxi_zone', 'total_public_transpo', 'walked', 'taxi',
       'total_car_van_truck', 'temp_sum', 'PUlocationID', 'new_taxi', 'FHV',
       'FHV_share'],
      dtype='object')

In [74]:
os.chdir('/Users/katharinevoorhees/CAPSTONE/')
lehd_adjust = pd.read_csv('taxi_zone_wages_raw.csv')
lehd_adjust.sort_values('PUlocationID').tail()

,DOlocationID,PUlocationID,LEHD_total_pair_count,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,median_household_income
18331,87.0,263.0,470,9.077847,8.344424,14.419385,12.315982,16.259863,20.079202,27.759392,69.868979,58.881463,91.844011,141.149452,89057.0
49433,232.0,263.0,22,0.424921,0.390590,0.674950,0.576493,0.761100,0.939878,1.299376,3.270463,2.756154,4.299081,6.606996,89057.0
18080,86.0,263.0,2,0.038629,0.035508,0.061359,0.052408,0.069191,0.085443,0.118125,0.297315,0.250559,0.390826,0.600636,89057.0
19311,91.0,263.0,8,0.154517,0.142033,0.245436,0.209634,0.276764,0.341774,0.472500,1.189259,1.002238,1.563302,2.402544,89057.0
56534,263.0,263.0,434,8.382523,7.705276,13.314922,11.372630,15.014427,18.541220,25.633141,64.517312,54.371393,84.809151,130.338005,89057.0


In [75]:
len(lehd_adjust.PUlocationID.unique())

260

In [76]:
limdf = lehd_adjust[lehd_adjust.PUlocationID != lehd_adjust.DOlocationID]

In [77]:
limdf.head()

,DOlocationID,PUlocationID,LEHD_total_pair_count,2500,7500,12500,17500,22500,30000,42500,62500,87500,125000,225000,median_household_income
1,3.0,4.0,1,0.070217,0.083930,0.116010,0.065062,0.063911,0.082528,0.103498,0.139483,0.068065,0.119664,0.087633,57788.0
2,3.0,5.0,2,0.044209,0.035588,0.066976,0.047745,0.048408,0.114279,0.224580,0.345491,0.212865,0.399867,0.459991,81420.5
3,3.0,6.0,1,0.050462,0.032627,0.046169,0.041018,0.038325,0.068415,0.087890,0.119658,0.118487,0.151192,0.245756,67361.0
4,3.0,7.0,7,0.257919,0.285195,0.333903,0.297581,0.234586,0.423943,0.596832,0.848628,0.724122,1.033718,1.963572,79656.0
5,3.0,9.0,3,0.096439,0.044976,0.077411,0.099034,0.095574,0.175580,0.358945,0.562635,0.358945,0.587286,0.543174,76912.0


In [34]:
OG = list(limdf.PUlocationID.unique())
OG_ = []
totals = []
for i in OG: 
    OG_.append(i)
    adding = limdf[limdf.PUlocationID == i]['LEHD_total_pair_count'].sum()
    totals.append(adding)
    
df = pd.DataFrame(totals, OG_).reset_index()
df.head()

,index,0
0,4.0,10617
1,5.0,8229
2,6.0,5269
3,7.0,36688
4,9.0,6229


In [78]:
new = new.merge(df, left_on='taxi_zone', right_on='index')

In [79]:
new.head()

,taxi_zone,total_public_transpo,walked,taxi,total_car_van_truck,temp_sum,PUlocationID,new_taxi,FHV,FHV_share,index,0
0,3.0,0.446290,0.054976,0.002905,0.495828,13424,3.0,0.000013,0.002050,0.000842,3.0,8706
1,4.0,0.653460,0.247877,0.025276,0.073387,10247,4.0,0.004036,0.013270,0.007969,4.0,10617
2,5.0,0.226116,0.004921,0.001867,0.767097,11786,5.0,0.000010,0.001599,0.000258,5.0,8229
3,6.0,0.374791,0.037601,0.001827,0.585782,6569,6.0,0.000021,0.001450,0.000356,6.0,5269
4,7.0,0.775097,0.073059,0.005607,0.146236,42445,7.0,0.000696,0.003887,0.001024,7.0,36688


In [80]:
cols = ['total_public_transpo', 'walked','new_taxi', 'FHV', 'FHV_share', 'total_car_van_truck']
for i in cols: 
    new[i] = new[i] * new[0]
new.head()

,taxi_zone,total_public_transpo,walked,taxi,total_car_van_truck,temp_sum,PUlocationID,new_taxi,FHV,FHV_share,index,0
0,3.0,3885.402712,478.622467,0.002905,4316.681764,13424,3.0,0.115434,17.843262,7.334361,3.0,8706
1,4.0,6937.780033,2631.714648,0.025276,779.153313,10247,4.0,42.851015,140.891180,84.609811,4.0,10617
2,5.0,1860.706347,40.495673,0.001867,6312.437553,11786,5.0,0.081377,13.158607,2.120444,5.0,8229
3,6.0,1974.772111,198.118892,0.001827,3086.483787,6569,6.0,0.109017,7.637848,1.878344,6.0,5269
4,7.0,28436.765508,2680.397880,0.005607,5365.117587,42445,7.0,25.522468,142.614028,37.582529,7.0,36688


In [82]:
new.rename(columns={'new_taxi':'P(mode1)','FHV':'P(mode2)','FHV_share':'P(mode3)'
                   ,'total_public_transpo':'P(mode4)','walked':'P(mode5)', 
                   'total_car_van_truck':'P(mode6)'},inplace=True)
new = new[['taxi_zone','P(mode1)','P(mode2)','P(mode3)','P(mode4)','P(mode5)', 'P(mode6)']]
new.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
0,3.0,0.115434,17.843262,7.334361,3885.402712,478.622467,4316.681764
1,4.0,42.851015,140.891180,84.609811,6937.780033,2631.714648,779.153313
2,5.0,0.081377,13.158607,2.120444,1860.706347,40.495673,6312.437553
3,6.0,0.109017,7.637848,1.878344,1974.772111,198.118892,3086.483787
4,7.0,25.522468,142.614028,37.582529,28436.765508,2680.397880,5365.117587


In [83]:
new.tail()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
238,259.0,0.458999,49.350904,19.972887,6172.661676,616.206964,5791.348569
239,260.0,17.568541,81.003213,23.376620,12114.056335,864.512997,4051.482293
240,261.0,9.275155,10.068801,1.332307,1650.124853,946.336663,89.862221
241,262.0,486.958660,276.121791,175.850061,9383.334176,2174.915351,1680.819962
242,263.0,424.285470,219.780064,106.859181,15349.517048,2684.573355,1349.984881


In [90]:
os.chdir('/Users/katharinevoorhees/Downloads/')
!ls

05-Oct-2018-1617 illegally parked bikes.gpx
05-Oct-2018-1624 smokers.gpx
10.1038_s41562-018-0521-2.ris
18-1038 295.jpg
2016CountyHealthIndicators.csv
2018-Urban-spatial-analytics_Lab01_creating_shapefiles (1).pdf
2018-Urban-spatial-analytics_Lab01_creating_shapefiles.pdf
2018-Urban-spatial-analytics_Lab02_spatial-joins.pdf
2018-Urban-spatial-analytics_Lab03_spatial-autocorrelation-analysis (1).pdf
2018-Urban-spatial-analytics_Lab03_spatial-autocorrelation-analysis.pdf
2018-Urban-spatial-analytics_Lecture04_spatial-clustering.pdf
311DataForClustering.csv
3_4_2019_Towards a .pdf
ACCESS Screening Analysis.ipynb
ACS_09_5YR_B02001
ACS_09_5YR_B02001.zip
ACS_09_5YR_B25077
ACS_09_5YR_B25077.zip
ACS_09_5YR_S1501
ACS_09_5YR_S1501.zip
ACS_14_5YR_B02001
ACS_14_5YR_B02001.zip
ACS_14_5YR_B25077
ACS_14_5YR_B25077.zip
ACS_14_5YR_S1501
ACS_14_5YR_S1501.zip
ACS_16_5YR_B01002
ACS_16_5YR_B01002.zip
ACS_17_1YR_GCT2801.ST50
ACS_17_1YR_GCT2801.ST50.zip
ACS_17_5YR_B01003.pdf
ACS_2016_5YR_TRACT_36_NEW_YORK.gdb

In [ ]:
!ls

In [91]:
new.to_csv('final_acs_transportation_choice.csv', index=False)

In [92]:
new.head()

,taxi_zone,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5),P(mode6)
0,3.0,0.115434,17.843262,7.334361,3885.402712,478.622467,4316.681764
1,4.0,42.851015,140.891180,84.609811,6937.780033,2631.714648,779.153313
2,5.0,0.081377,13.158607,2.120444,1860.706347,40.495673,6312.437553
3,6.0,0.109017,7.637848,1.878344,1974.772111,198.118892,3086.483787
4,7.0,25.522468,142.614028,37.582529,28436.765508,2680.397880,5365.117587
